<a href="https://colab.research.google.com/github/ashleykim00/ESAA/blob/main/11_8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **4. 모델 훈련 연습문제**


### 1) 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?

정답 :

보통은 확률적 경사 하강법(SGD), 미니배치 경사 하강법을 사용한다. 훈련 세트의 크기가 메모리에 맞다면 배치 경사 하강법도 가능하다. 

하지만 정규방정식은 계산 복잡도가 "특성의 갯수"에 따라 매우 빠르게 증가하기에 사용할 수 없다.



### 9) 릿지 회귀를 사용했을 때 훈련 오차와 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 α 를 증가시켜야할까요, 아니면 줄여야 할까요?

정답 :     

* 훈련 오차와 검증 오차가 비슷하므로, 단순한 모델이라고 유추할 수 있다.
* 단순한 모형의 경우 분산은 적지만, 편향이 크다. 
* 따라서, 높은 편향이 문제이다.
* 편향을 개선하기 위해서는 α 를 감소시켜야 한다.

-> 훈련 오차와 검증 오차가 비슷하다는 것을 과소적합되었다는 의미이므로 높은 편향이 문제이다.(편향이 높을수록 과소적합되기 쉽고 분산이 높을 수록 과대적합되기 쉽다.) 따라서 α를 감소시켜야한다.

교재 p.181, 186 쪽 참고)


### 10) 다음과 같이 사용해야 하는 이유는?

- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀

- 릿지 회귀 대신 라쏘 회귀

- 라쏘 회귀 대신 엘라스틱넷

정답 :    

- 규제가 약간 있는 것이 대부분의 경우 성능이 더 좋으므로, 일반적으로 평범한 선형 회귀는 피해야 한다.
- 릿지가 기본, 쓰이는 특성이 적으면 라쏘나 엘라스틱넷이 더 좋다.
- 특성 수가 훈련 샘플 수보다 많거나 특성 일부가 강하게 연관되어 있을경우, 보통 라쏘가 문제를 일으키므로 엘라스틱넷을 선호한다.

p.191 세 번째 줄~ 참고)



### 12) 다음 사이트에 들어가서 연습문제 12번 일부를 필사하세요.

https://github.com/rickiepark/handson-ml2/blob/master/04_training_linear_models.ipynb

- 67번 셀 부터 80번 셀 까지 필사하시면 됩니다.

- 시간이 남는다면 그 이후 코드까지 필사하셔도 좋습니다.

- 수식이 설명되어있는 부분을 보고싶으면 그 부분을 그대로 복사하여 텍스트 창에 붙여넣기 한 다음 직접 실행시키시면 확인해볼 수 있습니다.

In [2]:
from sklearn import datasets
iris = datasets.load_iris()

In [3]:
X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 넓이
y = iris["target"]

모든 샘플에 편향을 추가합니다 ($x_0 = 1$):

In [6]:
import numpy as np

In [7]:
X_with_bias = np.c_[np.ones([len(X), 1]), X]

In [8]:
np.random.seed(2042) # 결과를 일정하게 유지하기 위해 랜덤 시드를 지정

In [9]:
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]  # train_test_split() 원래는 사용하면 되지만 수동으로 데이터 나눠보자.

In [10]:
# 클래스 인덱스를 원-핫 벡터로 바꾸는 함수 작성
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [11]:
y_train[:10]  # 원-핫 함수 테스트 

array([0, 1, 2, 1, 1, 0, 1, 1, 1, 0])

In [12]:
to_one_hot(y_train[:10])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [13]:
# 훈련 세트와 테스트 세트의 타깃 클래스 확률을 담은 행렬 생성

Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [14]:
# 소프트맥스 함수 생성
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [16]:
X_train.shape[1]

3

In [17]:
len(np.unique(y_train))

3

In [15]:
# 입력과 출력의 개수를 정의
n_inputs = X_train.shape[1] # == 3 (특성 2개와 편향)
n_outputs = len(np.unique(y_train))   # == 3 (3개의 붓꽃 클래스)

In [18]:
# 비용함수 구현
# 소프트맥스 모델을 훈련

eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients

0 5.446205811872683
500 0.8350062641405651
1000 0.6878801447192402
1500 0.6012379137693314
2000 0.5444496861981872
2500 0.5038530181431525
3000 0.47292289721922487
3500 0.44824244188957774
4000 0.42786510939287936
4500 0.41060071429187134
5000 0.3956780375390374


In [19]:
Theta # 모델 파라미터

array([[ 3.32094157, -0.6501102 , -2.99979416],
       [-1.1718465 ,  0.11706172,  0.10507543],
       [-0.70224261, -0.09527802,  1.4786383 ]])

In [20]:
# 검증 세트에 대한 예측과 정확도를 확인
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9666666666666667

In [21]:
# l2 규제를 추가, 손실에 l2 패널티 추가, 그래디언트에도 항이 추가됨, 학습률 eta도 증가시킴
eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # 규제 하이퍼파라미터

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
        loss = xentropy_loss + alpha * l2_loss
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients
# 손실이 이전보다 커짐

0 6.629842469083912
500 0.5339667976629505
1000 0.5036400750148942
1500 0.49468910594603216
2000 0.4912968418075476
2500 0.48989924700933296
3000 0.4892990598451198
3500 0.48903512443978603
4000 0.4889173621830818
4500 0.48886433374493027
5000 0.48884031207388184


In [23]:
# 검증 세트에 대한 예측과 정확도를 확인, 정확도가 완벽함 
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

1.0